In [1]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import pprint
import string
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem import PorterStemmer
import numpy as np
np.random.seed(2018)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import string

ModuleNotFoundError: No module named 'gensim'

In [11]:
def read_file(path,filename):
    file = open(path+"/"+filename)
    text = word_tokenize(file.read())
    articles = []
    article = ""
    count = 0
    for msg in text:
        if len(msg) >4 and msg[:4] == '.LS.':
            if count != 0:
                articles.append(article.strip())
            else:
                count = 1
            article = ""
            
        else:
            if msg in string.punctuation:
                article += msg
            else:
                article += " "
                msg = wordnet_lemmatizer.lemmatize(msg.lower())
                article += msg
            
    articles.append(article.strip())    
    return articles

In [12]:
articles_4 = read_file('train_PURE','NS_LS_4_PURE.txt')

NameError: name 'wordnet_lemmatizer' is not defined

In [ ]:
articles_6 = read_file('/home/jovyan/work/CIS519/FinalProject/train_PURE','NS_LS_6_PURE.txt')

In [ ]:
articles_8 = read_file('/home/jovyan/work/CIS519/FinalProject/train_PURE','NS_LS_8_PURE.txt')

In [ ]:
articles_12 = read_file('/home/jovyan/work/CIS519/FinalProject/train_PURE','NS_LS_12_PURE.txt')

In [ ]:
articles_16 = read_file('/home/jovyan/work/CIS519/FinalProject/train_PURE','SA_LS_PURE.txt')

In [ ]:
def prepare_dataset(list_articles,list_lables):
    X_train = []
    y_train = []
    college_count = 0
    for x in list_articles:
        print(len(x))
    for i in range(300):
        
        #append grade 4
        X_train.append(list_articles[0][i])
        y_train.append(list_lables[0])
        #append grade 6
        X_train.append(list_articles[1][i])
        y_train.append(list_lables[1])
        #append grade 8
        X_train.append(list_articles[2][i])
        y_train.append(list_lables[2])
        #append grade 12
        X_train.append(list_articles[3][i])
        y_train.append(list_lables[3])
        if ((i+1) // 3) != 0 and college_count < 200:
            #append college
            X_train.append(list_articles[4][college_count])
            y_train.append(list_lables[4])
            college_count += 1
    n = len(y_train)
    print(n)
    y_train = np.array(y_train)
    y_train = y_train.reshape((n,))
    return (np.asarray(X_train),y_train)

In [ ]:
list_articles = [articles_4,articles_6,articles_8,articles_12,articles_16]
list_lables = [4,6,8,12,16]

In [ ]:
X_train, y_train = prepare_dataset(list_articles,list_lables)

In [ ]:
stop_words = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def my_kernel(X, Y):
    return cosine_similarity(X,Y)

In [ ]:
def read_document_null(file):
    text = file.read()
    sentences = sent_tokenize(text)
    total_string = ""
    for sent in sentences:
        words = word_tokenize(sent)
        string = "" 
        for word in words:
            if re.search('^[a-zA-Z]+$', word):
                word = wordnet_lemmatizer.lemmatize(word.lower())
                if word not in stop_words:
                    string += word
                    string += " "
        string = string.strip()
        if len(string)> 0:
            total_string += string + " "            
    return total_string.strip()

In [ ]:
def cross_validation(k_fold,clf,train_X,train_y,max_feature,ngram_range):
    accuracy_list = []
    precision_list = []
    recall_list = []
    for train, test in k_fold.split(train_X):
        vect = TfidfVectorizer(max_features = max_feature,stop_words = stop_words)
        #vect = TfidfVectorizer(stop_words = stop_words)
        train_X_vec = vect.fit_transform(train_X[train]).toarray()
        clf.fit(train_X_vec, train_y[train])
        newVec = TfidfVectorizer(vocabulary=vect.vocabulary_,max_features = max_feature,ngram_range = ngram_range,stop_words = stop_words)
        #newVec = TfidfVectorizer(vocabulary=vect.vocabulary_,stop_words = stop_words)
        test_X_vec = newVec.fit_transform(train_X[test]).toarray()
        y_pred = clf.predict(test_X_vec)
        y_true = train_y[test]
        accuracy_list.append(accuracy_score(y_true, y_pred))
        precision_list.append(precision_score(y_true, y_pred,y_pred,average = 'macro'))
        recall_list.append(recall_score(y_true, y_pred,average = 'macro'))
    result = []    
    for score in [accuracy_list,precision_list,recall_list]:
        result.append(np.mean(score))
    return result

In [ ]:
def train_models(train_X, train_y,max_feature,ngram_range):
    k_fold = KFold(n_splits=5,shuffle = True)
    results = []
    models = []
    models.append(svm.SVC(C=0.01, kernel=my_kernel))
    models.append(GaussianNB())
    models.append(LogisticRegression(random_state=0, multi_class='multinomial',solver = 'saga'))
    models_name = ['SVM-cosine','GaussianNB','MLR']
    
    for i in range(len(models)):
        print(i)
        lis = cross_validation(k_fold,models[i],train_X, train_y,max_feature,ngram_range)
        results.append([models_name[i],lis[0],lis[1],lis[2]])
    results_df = pd.DataFrame(results)
    results_df.columns = ['MODEL', 'Accuracy','Precision','Recall']
    return results_df
    

In [ ]:
print()

In [ ]:
#stop_word 
train_models(X_train, y_train,1000,(1,1))

In [ ]:
#stop_word + lemmatization
train_models(X_train, y_train,300,(1,1))